I only includes changes here so that it is easier to merge

In [ ]:
prefix = 'data_ACSF_SOAP_atomInfo_otherInfo_QM9'

### QM9 at mol level

In [4]:
with open('../Data/qm9_mol.pickle', 'rb') as handle:
    qm9_mol = pickle.load(handle)

#### Add {'u':qm9_mol[m]}

In [11]:
def create_data(mols,IsTrain):
    type_list = [[] for _ in range(8)]
    tot_list = []
    if not IsTrain:
        test_id_type_list = [[] for _ in range(8)]
        test_id_list = []
        
    for m in mols:
        if IsTrain:
            dict_ = {'x':np.concatenate([structures_dict[m],structures_dict2[m],structures_dict3[m]],1),\
                       'edge_index':bonds_edge_index[m],\
                       'edge_attr':bonds_edge_attr[m],'y':coupling_y[m],\
                       'edge_index3':coupling_edge_index[m],'edge_attr3':coupling_edge_attr[m],\
                       'edge_attr4':coupling_edge_dist[m],\
                       'y_mol':mol_info[m],'y_atom':atom_info[m],'y_coupling':coupling_info[m],\
                       'u':qm9_mol[m]}
            tot_list.append(copy.deepcopy(dict_))
                        
            temp = dict_['edge_attr3'].argmax(1)
            for i in np.nonzero(dict_['edge_attr3'].sum(0))[0]:
                dict_['type_attr'] = (temp==i).astype(np.uint8)
                type_list[i].append(copy.deepcopy(dict_))
        else:
            dict_ = {'x':np.concatenate([structures_dict[m],structures_dict2[m],structures_dict3[m]],1),\
                       'edge_index':bonds_edge_index[m],\
                       'edge_attr':bonds_edge_attr[m],\
                       'edge_index3':coupling_edge_index[m],'edge_attr3':coupling_edge_attr[m],\
                       'edge_attr4':coupling_edge_dist[m],\
                       'u':qm9_mol[m]}
            tot_list.append(copy.deepcopy(dict_))
            test_id_list.append(coupling_id[m])
            
            temp = dict_['edge_attr3'].argmax(1)
            for i in np.nonzero(dict_['edge_attr3'].sum(0))[0]:
                dict_['type_attr'] = (temp==i).astype(np.uint8)
                type_list[i].append(copy.deepcopy(dict_))
                test_id_type_list[i].append(coupling_id[m][temp==i])
    
    if IsTrain:
        return tot_list,type_list 
    else:
        return tot_list,type_list,np.concatenate(test_id_list),[np.concatenate(type_i) for type_i in test_id_type_list]